Link to the dataset: http://www.nlpr.ia.ac.cn/pal/trafficdata/recognition.html

In [1]:
import numpy as np
import pandas as pd
import cv2
from skimage import color, exposure, transform

In [2]:
from skimage import io
import os
import glob

In [3]:
from keras.optimizers import SGD
from cnn import cnn

Using TensorFlow backend.


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
img_size = 50

In [6]:
def preprocess_img(img, img_size):
    # Histogram normalization
    hsv = color.rgb2hsv(img)
    hsv[:, :, 2] = exposure.equalize_hist(hsv[:, :, 2])
    img = color.hsv2rgb(hsv)

    # central square crop
    min_side = min(img.shape[:-1])
    centre = img.shape[0] // 2, img.shape[1] // 2
    img = img[centre[0] - min_side // 2:centre[0] + min_side // 2,
              centre[1] - min_side // 2:centre[1] + min_side // 2,:]

    # rescale to standard size
    img = transform.resize(img, (img_size, img_size))

    # roll color axis to axis 0
    img = np.rollaxis(img, -1)
    
    return img

In [7]:
def get_class(img_path):
    return int(img_path.split('/')[-1].split('_')[0])

root_dir = 'chn/train_images'
imgs = []
labels = []

all_img_paths = glob.glob(os.path.join(root_dir, '*'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path), img_size)
    label = get_class(img_path)
    imgs.append(img)
    labels.append(label)

num_classes = len(set(labels))
X = np.array(imgs, dtype='float32')
# Make one hot targets
Y = np.eye(num_classes, dtype='uint8')[labels]

In [8]:
Y.shape

(4170, 58)

In [9]:
X.shape

(4170, 3, 50, 50)

In [10]:
model = cnn(img_size,num_classes)

# let's train the model using SGD + momentum
lr = 0.01
sgd = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [11]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

#Learning rate scheduler
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

batch_size = 32
epochs = 15

model.fit(X, Y,
          batch_size=batch_size,
          epochs=epochs,
          #validation_split=0.2,
          callbacks=[LearningRateScheduler(lr_schedule),
                     ModelCheckpoint('model.h5', save_best_only=True)]
          )

Epoch 1/15
4170/4170 [==============================] - 81s 19ms/step - loss: 3.6757 - acc: 0.0942
Epoch 2/15
4170/4170 [==============================] - 114s 27ms/step - loss: 2.6049 - acc: 0.3331
Epoch 3/15
4170/4170 [==============================] - 134s 32ms/step - loss: 1.0642 - acc: 0.7000
Epoch 4/15
4170/4170 [==============================] - 89s 21ms/step - loss: 0.6120 - acc: 0.8199
Epoch 5/15
4170/4170 [==============================] - 127s 30ms/step - loss: 0.4408 - acc: 0.8736
Epoch 6/15
4170/4170 [==============================] - 84s 20ms/step - loss: 0.2764 - acc: 0.9192
Epoch 7/15
4170/4170 [==============================] - 132s 32ms/step - loss: 0.2267 - acc: 0.9355
Epoch 8/15
4170/4170 [==============================] - 75s 18ms/step - loss: 0.1642 - acc: 0.9513
Epoch 9/15
4170/4170 [==============================] - 77s 18ms/step - loss: 0.1471 - acc: 0.9552
Epoch 10/15
4170/4170 [==============================] - 96s 23ms/step - loss: 0.1742 - acc: 0.9513
Epoch

In [13]:
root_dir = 'chn/test_images'
imgs_test = []
labels_test = []

all_img_paths = glob.glob(os.path.join(root_dir, '*'))
np.random.shuffle(all_img_paths)
for img_path in all_img_paths:
    img = preprocess_img(io.imread(img_path), img_size)
    label = get_class(img_path)
    imgs_test.append(img)
    labels_test.append(label)

X_test = np.array(imgs_test, dtype='float32')
y_test = np.array(labels_test)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
# predict 
pred = model.predict_classes(X_test)

In [16]:
pred

array([ 5, 13,  2, ..., 46, 16, 27])

In [17]:
accuracy_score(pred,labels_test)

0.7331995987963892